ONET Task Similarity Dataset
---

By Paul Duckworth 20th Sept 2017.

Create a Task Similarity Matrix from ONET datasets of Tasks, DWAs, IWAs and WAs. 

Use Future of Work survey as Ground Truth in 1D Gaussian Process to infer over all (DWA) Tasks. 


In [1]:
import os
import numpy as np
import pandas as pd
import getpass
import pylab as plt
import cPickle as pickle
from random import shuffle
import scipy as sp
import scipy.optimize
%matplotlib inline

datasets = '/home/'+ getpass.getuser() +'/Datasets/'
print datasets


/home/scpd/Datasets/


## ONET TASK data:


In [2]:
tasks = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/Task Statements.txt'), sep='\t')
tasks = tasks[['Task ID', 'Task']]
#tasks = tasks.loc[range(20)]     # reduce the task matrix for now :)
tasks.shape

(19566, 2)

In [3]:
#Task DWAs (detailed work activitiy code):
taskDWA = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/Tasks to DWAs.txt'), sep='\t')
taskDWA = taskDWA[['Task ID', 'DWA ID']]

print taskDWA.shape, "UNIQUE DWA: ", len(taskDWA['DWA ID'].unique()) 
print taskDWA.head()

(22838, 2) UNIQUE DWA:  2070
   Task ID             DWA ID
0    20461  4.A.2.a.4.I09.D03
1    20461  4.A.4.b.6.I08.D04
2     8823  4.A.4.b.4.I09.D02
3     8824  4.A.4.a.2.I03.D14
4     8825  4.A.2.a.4.I07.D09


In [4]:
df = pd.merge(tasks, taskDWA,  how='left', left_on=['Task ID'], right_on = ['Task ID']).sort_values(by = 'Task ID')
df = df[df['DWA ID'].notnull()]
df['IWA ID'] = df['DWA ID'].str.slice(0,-4)    # create IWA ID
df['WA ID'] = df['DWA ID'].str.slice(0,-8)     # create WA ID

## ADD DWA and IWA titles:
DWAref = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/DWA Reference.txt'), sep='\t')[['DWA ID', 'DWA Title']]
df2 = pd.merge(df, DWAref,  how='left', left_on=['DWA ID'], right_on = ['DWA ID'])

IWAref = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/IWA Reference.txt'), sep='\t')[['IWA ID', 'IWA Title']]
df3 = pd.merge(df2, IWAref,  how='left', left_on=['IWA ID'], right_on = ['IWA ID'])

cols = ['Observed Occupation', 'O*NET Occupation title', 'O*NET-SOC Code','Description']
df3[['Task ID', 'Task', 'DWA ID', 'DWA Title', 'IWA ID', 'IWA Title', 'WA ID']]

print df3.shape
df3.head()

(22838, 7)


,Task ID,Task,DWA ID,IWA ID,WA ID,DWA Title,IWA Title
0,1,Resolve customer complaints regarding sales an...,4.A.4.a.8.I03.D05,4.A.4.a.8.I03,4.A.4.a.8,Resolve customer complaints or problems.,Respond to customer problems or inquiries.
1,2,Monitor customer preferences to determine focu...,4.A.1.a.1.I14.D04,4.A.1.a.1.I14,4.A.1.a.1,Conduct opinion surveys or needs assessments.,Collect data about consumer needs or opinions.
2,3,Direct and coordinate activities involving sal...,4.A.4.b.4.I12.D03,4.A.4.b.4.I12,4.A.4.b.4,"Direct sales, marketing, or customer service a...","Direct organizational operations, activities, ..."
3,4,Determine price schedules and discount rates.,4.A.2.b.4.I01.D06,4.A.2.b.4.I01,4.A.2.b.4,Determine pricing or monetary policies.,"Develop organizational policies, systems, or p..."
4,5,Review operational records and reports to proj...,4.A.2.a.4.I11.D06,4.A.2.a.4.I11,4.A.2.a.4,Analyze financial records or reports to determ...,Analyze business or financial data.


In [5]:
# Every IWA is linked to exactly one WA from the O*NET Content Model. 
# IWAs are linked to one or more DWAs; 

WA = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/Work Activities.txt'), sep='\t')
WA.rename(columns = {'Element ID':'WA ID', 'Element Name':'WA Title'}, inplace = True)
# WA[['WA IM Value', 'WA IM SE']] = WA[['Data Value', 'Standard Error']]  # This is per Occupation. 

WA = WA[WA['Scale ID'] == "IM"][['WA ID', 'WA Title']] #, 'WA IM Value', 'WA IM SE']]
WA.drop_duplicates(inplace=True)
print WA.shape
WA.head()

(41, 2)


,WA ID,WA Title
0,4.A.1.a.1,Getting Information
2,4.A.1.a.2,"Monitor Processes, Materials, or Surroundings"
4,4.A.1.b.1,"Identifying Objects, Actions, and Events"
6,4.A.1.b.2,"Inspecting Equipment, Structures, or Material"
8,4.A.1.b.3,Estimating the Quantifiable Characteristics of...


In [6]:
df4 = pd.merge(df3, WA, how='left', left_on=['WA ID'], right_on = ['WA ID']).sort_values(by = 'Task ID')
# df4[df4['Task ID'].notnull()]
print df4.shape

#df4[df4['IWA ID'].str.contains('4.A.4.b.4')].drop_duplicates(subset=['IWA ID'])#.sort_values(by = 'IWA ID')
df4.head()

(22838, 8)


,Task ID,Task,DWA ID,IWA ID,WA ID,DWA Title,IWA Title,WA Title
0,1,Resolve customer complaints regarding sales an...,4.A.4.a.8.I03.D05,4.A.4.a.8.I03,4.A.4.a.8,Resolve customer complaints or problems.,Respond to customer problems or inquiries.,Performing for or Working Directly with the Pu...
1,2,Monitor customer preferences to determine focu...,4.A.1.a.1.I14.D04,4.A.1.a.1.I14,4.A.1.a.1,Conduct opinion surveys or needs assessments.,Collect data about consumer needs or opinions.,Getting Information
2,3,Direct and coordinate activities involving sal...,4.A.4.b.4.I12.D03,4.A.4.b.4.I12,4.A.4.b.4,"Direct sales, marketing, or customer service a...","Direct organizational operations, activities, ...","Guiding, Directing, and Motivating Subordinates"
3,4,Determine price schedules and discount rates.,4.A.2.b.4.I01.D06,4.A.2.b.4.I01,4.A.2.b.4,Determine pricing or monetary policies.,"Develop organizational policies, systems, or p...",Developing Objectives and Strategies
4,5,Review operational records and reports to proj...,4.A.2.a.4.I11.D06,4.A.2.a.4.I11,4.A.2.a.4,Analyze financial records or reports to determ...,Analyze business or financial data.,Analyzing Data or Information


## obtain GTs from survey on Future of Employment

In [7]:
survey_data = pd.read_csv(os.path.join(datasets, 'FoEmployment/fow-expert-survey/data/cleaned/counts_data_with_metadata.csv'))
survey_data.rename(columns = {'title':'O*NET Occupation title', 
                              'Unnamed: 0': 'Task'}, inplace = True)

# Change Ordinal Data to Numeric - bit hacky
ratings = [4,3,2,1,0]
survey_data['GT Rating'] = (survey_data['Completely Automatable Today']*ratings[0] + survey_data['Could be Mostly Automated Today (Human Still Needed)']*ratings[1] + survey_data['Mostly Not Automatable Today (Human Does Most of It)']*ratings[2] + survey_data['Not Automatable Today']*ratings[3] + survey_data['Unsure']*ratings[4]) / survey_data['Number of Responses']                
                
survey_data = survey_data[['Task ID', 'GT Rating']]
print survey_data.count()
# survey_data.head()

Task ID      350
GT Rating    350
dtype: int64


In [8]:
# Do tasks overlap between occus? No probs not. merge this in... 
print survey_data.shape
print len(survey_data['Task ID'].unique())
# survey_data[survey_data.duplicated(subset='Task ID', keep=False) == True]


(350, 2)
348


In [9]:
task_dwa_rat = pd.merge(taskDWA, survey_data, how='left', left_on=['Task ID'], right_on = ['Task ID'])#.sort_values(by = 'Task ID')

print task_dwa_rat.shape
print "Instances of annotated DWA IDs: ", sum(task_dwa_rat['GT Rating'].notnull())   # boolean 
# task_dwa_rat[task_dwa_rat['GT Rating'].notnull()].head()

# task_dwa_rat[task_dwa_rat['GT Rating'].notnull()][task_dwa_rat['DWA ID'] == '4.A.4.b.4.I12.D39']

(22840, 3)
Instances of annotated DWA IDs:  416


In [10]:
DWA_mean_rating = task_dwa_rat.groupby(['DWA ID']).mean().reset_index().rename(columns = {'GT Rating':'DWA GT Rating'})
DWA_mean_rating.head() #[DWA_mean_rating['DWA ID'] == '4.A.4.b.4.I12.D39']

print "Unique DWAs Annotated = ", DWA_mean_rating[DWA_mean_rating['DWA GT Rating'].notnull()].shape

Unique DWAs Annotated =  (314, 3)


# DWA Level Dataset 

In [11]:
#consistency check: how many DWAs are mapped up from Task to DWA (all 2070 :)
print len(DWA_mean_rating['DWA ID'].unique())

keep_columns = ['DWA ID', 'DWA GT Rating', 'IWA ID', 'WA ID', 'DWA Title', 'IWA Title', 'WA Title']
df5 = pd.merge(df4, DWA_mean_rating, how='left', left_on=['DWA ID'], right_on = ['DWA ID'])[keep_columns].drop_duplicates(subset=keep_columns)
print df5.shape
# df5[df5['DWA GT Rating'].notnull()].head()

df5.head()


2070
(2070, 7)


,DWA ID,DWA GT Rating,IWA ID,WA ID,DWA Title,IWA Title,WA Title
0,4.A.4.a.8.I03.D05,NaN,4.A.4.a.8.I03,4.A.4.a.8,Resolve customer complaints or problems.,Respond to customer problems or inquiries.,Performing for or Working Directly with the Pu...
1,4.A.1.a.1.I14.D04,NaN,4.A.1.a.1.I14,4.A.1.a.1,Conduct opinion surveys or needs assessments.,Collect data about consumer needs or opinions.,Getting Information
2,4.A.4.b.4.I12.D03,NaN,4.A.4.b.4.I12,4.A.4.b.4,"Direct sales, marketing, or customer service a...","Direct organizational operations, activities, ...","Guiding, Directing, and Motivating Subordinates"
3,4.A.2.b.4.I01.D06,NaN,4.A.2.b.4.I01,4.A.2.b.4,Determine pricing or monetary policies.,"Develop organizational policies, systems, or p...",Developing Objectives and Strategies
4,4.A.2.a.4.I11.D06,NaN,4.A.2.a.4.I11,4.A.2.a.4,Analyze financial records or reports to determ...,Analyze business or financial data.,Analyzing Data or Information


# Split into Train, Validate and Test set

In [12]:
data = df5.reset_index()
X = data[data['DWA GT Rating'].notnull()].reset_index()
test = df5[df5['DWA GT Rating'].isnull()].reset_index()
y = X['DWA GT Rating']

In [13]:
save_this = (X, test, y)
file_name = 'tasks_by_similarity.p'
f = open(os.path.join(datasets, 'FoEmployment/Analysis_of_ONET_Tasks', file_name), "w")
pickle.dump(save_this, f)
f.close()

In [14]:
val_size = 0.1

# Create a Validation set
msk = [i for i in range(X.shape[0])]
shuffle(msk)

n_validation_set = int(np.floor(X.shape[0]*val_size))
print msk[:n_validation_set]

x_train = X.iloc[msk[n_validation_set:]].reset_index(drop=True)
x_valid = X.iloc[msk[:n_validation_set]].reset_index(drop=True)

y_train = y[msk[n_validation_set:]].as_matrix().reshape(x_train.shape[0], 1)
y_valid = y[msk[:n_validation_set]].as_matrix().reshape(n_validation_set, 1)

x_test = test.reset_index(drop=True)

# m = learn_a_GP(x_train, y_train, k)
# y_pred, Vp = m.predict(x_valid, full_cov=False)
# err = rmse(y_valid, y_pred)
# RMS_reps.append(err)
# print "RMS Error is: " + str(err)

[307, 227, 110, 178, 268, 152, 305, 58, 56, 184, 155, 236, 100, 163, 237, 187, 68, 90, 70, 92, 125, 88, 23, 301, 132, 87, 113, 30, 191, 161, 41]


In [15]:
print X.shape
print x_train.shape
print y_train.shape
print x_valid.shape
print y_valid.shape
print x_test.shape

(314, 9)
(283, 9)
(283, 1)
(31, 9)
(31, 1)
(1756, 8)


In [45]:
def similarity_kernel(A, B, args):
    K = np.zeros([A.shape[0], B.shape[0]])
    
    for index, row in A.iterrows(): 
        matchesIWA = row['IWA ID'] == B['IWA ID']
        matchesWA = row['WA ID'] == B['WA ID']
        noMatch = row['IWA ID'] != B['IWA ID']
        update_cov_row = matchesIWA*args[0] + matchesWA*args[1] + noMatch*args[2]  # operators on bool vectors works fine
        K[index] += update_cov_row
    return K # ((K - K.min(axis=0)) / (K.max(axis=0) - K.min(axis=0)))

# why doesnt unit normalising the Kernels help? 


def gaussian_process(K, y, Ks = None, Kss = None, predict = False):
    N = K.shape[0]
    jitter = 1e-6

    # compute the Posterior distribution (mean and covariance)
    diag = jitter*np.eye(N)
    flag = False
    while flag == False:
        try: 
            L = np.linalg.cholesky(K+diag)
            flag = True
        except np.linalg.LinAlgError as e:
            print ".",
            diag = diag + jitter*np.eye(N)

    # # solve for m where: L*m = y
    m = np.linalg.solve(L, y)

    # # solve for alpha where: L.T*alpha = m
    alpha = np.linalg.solve(L.T, m)

    LML = -0.5*np.dot(y.T, alpha) - sum(np.log(np.diag(L))) - 0.5*N*np.log(2*np.pi) # larger (negative) better
    
    if predict:
        print "predicting... "
        # compute the posterior mean for test points Ks
        mu = np.dot(Ks.T, alpha)

        # compute the variance at our test points
        # solve for v where: Lv = Kstar
        v = np.linalg.solve(L, Ks)  

        var = np.diag(Kss) - np.sum(v**2, axis=0)
        std = np.sqrt(var)

        return mu, std, LML
    return LML


def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())


# Optimisation

need to add this into a cross validation loop:

- randomise the validation set each loop, 

- optimise on the trainig each loop,

- evaluate RMSE each loop. :) 


In [26]:
def f_to_optimise(hyp, *args):
    sigma = hyp[-1]
    theta = (hyp[0], hyp[1], hyp[2])
    (x_train, y_train) = args
    K = similarity_kernel(x_train, x_train, theta) +  sigma**2*np.eye(x_train.shape[0])   # added some gaussian Noise
    LML = gaussian_process(K, y_train)
    return LML[0]


In [46]:
hyp = (0.8, 0.4, 0.1, 1e-1)    # Initial Guess: Theta and noise sigma
print "args = ", hyp 
LML = f_to_optimise(hyp, x_train, y_train)
print "LML = ", LML

args =  (0.8, 0.4, 0.1, 0.1)
LML =  [-1411.87487312]


In [43]:
hyps = np.array((15, 5, 1., 1e-1)).reshape(4,1)   # when xval, restart this randomly throughout space 
bnds =((0.1, None), (0.1, None), (0.01, None), (0.01, None))
args = (x_train, y_train)

hyp_opt = scipy.optimize.minimize(f_to_optimise, hyps, args = args,  method='L-BFGS-B', bounds=bnds, options={'maxiter':1000})
print hyp_opt

print "Otimisation: done" 

      fun: array([-132787.76683306])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  6.02478394e+01,   2.71247700e+00,   4.20986908e+01,
         2.62786204e+07])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 120
      nit: 1
   status: 2
  success: False
        x: array([  2.00888048e+01,   6.68799943e+00,   1.00000000e-02,
         1.00000000e-02])
Otimisation: done


In [21]:
print "hello world"

hello world


In [37]:
hyp = hyp_opt.x
print "\nargs = ", hyp 
LML = f_to_optimise(hyp, x_train, y_train)
print "LML = ", LML


args =  [ 0.1   0.1   0.01  0.01]
LML =  [-132798.26483154]


In [44]:
## Compute the predicted y for validation set: 

# hyps = np.array((11, 1, 0.1, 1e-1)).reshape(4,1)
hyp = hyp_opt.x

sigma = hyp[-1]
theta = (hyp[0], hyp[1] , hyp[2])

K = similarity_kernel(x_train, x_train, theta) +  sigma**2*np.eye(x_train.shape[0])   # added some gaussian Noise
Ks = similarity_kernel(x_train, x_valid, theta)
Kss = similarity_kernel(x_valid, x_valid, theta)

mu, std, LML = gaussian_process(K, y_train, Ks, Kss, predict = True)

for cnt, (index, row), in enumerate(x_valid.iterrows()):
    print "mean = %0.3f, std = %0.4f. Actual = %0.3f" % (mu[cnt], std[cnt], y_valid[cnt][0])

err = rmse(y_valid, mu)
print "RMS Error is: " + str(err)

# for cnt, (index, row), in enumerate(x_valid.iterrows()):
#     print "mean = %0.3f, std = %0.4f. DWA = %s" % (mu[cnt], std[cnt], row['DWA Title'])
# print "\nThis seems OK: ", max(mu)

# N = K.shape[0]
# LML = -0.5*np.dot(y.values.T, alpha) - sum(np.log(np.diag(L))) - 0.5*N*np.log(2*np.pi) # larger (negative) better
# print LML



predicting... 
mean = 2.385, std = 0.0058. Actual = 2.400
mean = 3.519, std = 0.0071. Actual = 2.312
mean = 2.561, std = 0.0100. Actual = 2.500
mean = 1.929, std = 0.0071. Actual = 2.333
mean = 1.837, std = 0.0058. Actual = 2.091
mean = 1.989, std = 4.7904. Actual = 3.167
mean = 1.698, std = 4.7234. Actual = 2.357
mean = 1.356, std = 4.9088. Actual = 2.727
mean = 2.810, std = 0.0071. Actual = 3.182
mean = 2.342, std = 0.0058. Actual = 2.800
mean = 2.648, std = 0.0045. Actual = 1.333
mean = 1.958, std = 4.6802. Actual = 2.154
mean = 2.412, std = 0.0100. Actual = 3.417
mean = 2.500, std = 0.0071. Actual = 2.333
mean = 2.186, std = 0.0071. Actual = 1.731
mean = 3.400, std = 0.0071. Actual = 2.778
mean = 1.356, std = 4.9088. Actual = 2.545
mean = 3.900, std = 0.0100. Actual = 3.700
mean = 1.693, std = 0.0071. Actual = 3.000
mean = 2.154, std = 4.6639. Actual = 3.283
mean = 1.837, std = 0.0058. Actual = 2.125
mean = 2.667, std = 0.0100. Actual = 3.667
mean = 2.154, std = 4.6639. Actual = 2.

# Sanity Checks: 

In [ ]:
# Top 10 most and Least Automatable Inferred Task Ratings: 
appended_list = []
for cnt, (index, row), in enumerate(test.iterrows()):
    if row['WA ID'] in ['4.A.4.a.4','4.A.3.b.4']:   # These two WA's don't have any training data
        continue
    appended_list.append((mu[cnt], std[cnt], row['DWA Title'], row['WA ID']))

# for i in [(y[0], y[3], y[2]) for y in sorted(appended_list, key=lambda x: x[0])[:10]]:
#     print i

# print "\n"
# for i in [(y[0], y[3], y[2]) for y in sorted(appended_list, key=lambda x: x[0])[-10:]]:
#     print i  

In [ ]:
# 2 WA's are not represented in the training data - so they come out with 0 Automatability. 
print '4.A.3.b.4' in  X['WA ID'].unique()
print '4.A.4.a.4' in  X['WA ID'].unique()

print len(X['WA ID'].unique())
print len(Xtest['WA ID'].unique())
print len(df5['WA ID'].unique())

In [ ]:
print "Training Set: "
print "number of DWAs per IWA in training data = %0.3f" % X.groupby(['IWA ID']).count()['DWA ID'].mean()
# print "number of IWAs per WA in training data = %0.3f" % X.drop_duplicates(subset=['WA ID', 'IWA ID']).groupby(['WA ID']).count()['IWA ID'].mean()
# print "number of DWAs per WA in training data = %0.3f" % X.drop_duplicates(subset=['WA ID', 'IWA ID', 'DWA ID']).groupby(['WA ID']).count()['DWA ID'].mean()

# add negative penalties for sharing IWA 

print "\nTest Set: "
print "number of DWAs per IWA in test data = %0.3f" % Xtest.groupby(['IWA ID']).count()['DWA ID'].mean()
# print "number of IWAs per WA in test data = %0.3f" % Xtest.drop_duplicates(subset=['WA ID', 'IWA ID']).groupby(['WA ID']).count()['IWA ID'].mean()
# print "number of DWAs per WA in test data = %0.3f" % Xtest.drop_duplicates(subset=['WA ID', 'IWA ID', 'DWA ID']).groupby(['WA ID']).count()['DWA ID'].mean()

# print "\n"
# print X.groupby(['IWA ID']).count()['DWA ID']
# print Xtest.groupby(['IWA ID']).count()['DWA ID']

